## Analyzing 2015-19 ACS Data 

This notebook contains the code used to wrangle, join, and analyze the following datasets:

+ American Community Survey (ACS) 5-year estimates (2015-2019)
    + [Aggregate](https://api.census.gov/data/2019/acs/acs5/variables.html)
    + [Profile](https://api.census.gov/data/2019/acs/acs5/profile/variables.html)

+ Chicago Community Area Census Tract Crosswalk
    + [Chicago Community Areas](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6)
    + [Chicago Census Tracts](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Census-Tracts-2010/5jrd-6zik)
+ [Chicago COVID-19 Community Vulnerability Index (CCVI)](https://data.cityofchicago.org/Health-Human-Services/Chicago-COVID-19-Community-Vulnerability-Index-CCV/xhc6-88s9)
+ [Hardship Index](https://data.cityofchicago.org/Health-Human-Services/hardship-index/792q-4jtu)



All csv files can be found in the "data" folder of the [working](https://github.com/danielgrzenda/broadbandequity/tree/working) branch of our Broadband Equity Github repo.

##### Importing Libraries

In [1]:
import os
import sys
sys.path[0] = os.path.join(os.path.abspath(''),'..')  # make sure we can import from our package

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from data_pipeline import fetch_census_data

In [4]:
#fetch_census_data.acs5_profile(force_api_call=True)

##### Importing Chicago ACS aggregate and profile data

In [5]:
# ACS aggregate

acs_agg = fetch_census_data.acs5_aggregate().drop(['state', 'county'], axis=1)

In [6]:
# ACS profile

acs_pro = fetch_census_data.acs5_profile().drop(['state', 'county'], axis=1)

In [7]:
# merging both ACS datasets
# renaming variables

acs_df = acs_agg.merge(acs_pro, on='tract').rename(columns={
            'estimated total population': 'total_pop',
            'estimated total households': 'total_households',
            'estimated total with internet subscription': 'hh_internet',
            'estimated total with no internet access': 'hh_no_internet',
            'estimated total has no computer' : 'hh_no_computer',
            'estimated total households with a computer' : 'hh_computer',
            'estimated total households with broadband internet subscription': 'hh_broadband',
            'estimated total households median household income': 'hh_median_income',
            'percent estimated percent of families and people whose income in the past 12 months is below poverty level': 'hh_poverty(%)',
            'percent estimated unemployment': 'hh_unemployment(%)',
            'estimated total population hispanic or latino (of any race)': 'total_hispanic',
            'estimated total population Black or African American alone (non-Hispanic)': 'total_black',
            'estimated total population Asian (non-Hispanic)' : "total_asian",
            'estimated total population White (non-Hispanic)' : "total_white"})


In [8]:
# removing tracts outside of city boundaries
# Ohare tracts

acs_df = acs_df[acs_df['tract'] != 770602]
acs_df = acs_df[acs_df['tract'] != 770902]
acs_df = acs_df[acs_df['tract'] != 980000]

In [9]:
# 1316 rows x 13 columns
# variables beginning with "hh" indicate they are measured at the household level
# variables with "(%)" indicate they are recorded as percentages 

acs_df.head(5)

,total_pop,hh_internet,hh_no_internet,hh_no_computer,tract,total_households,hh_computer,hh_broadband,hh_median_income,hh_poverty(%),total_hispanic,total_black,hh_unemployment(%),total_white,total_asian
0,1825,392,149,149,630200,575,426,392,37422,25.7,1622,0,11.7,125,78
1,5908,1242,231,133,580700,1544,1411,1242,47000,17.4,4742,161,13.1,423,522
2,3419,928,140,104,590600,1172,1068,917,46033,7.9,2119,9,8.1,757,431
3,2835,917,138,81,600700,1084,1003,917,45294,17.0,850,82,11.8,1001,857
4,1639,322,245,218,611900,574,356,322,24507,55.0,438,1175,21.1,26,0


##### Importing Chicago COVID and Hardship Indices

In [10]:
# Chicago COVID hardship index csv

covid = pd.read_csv("../data/covid_index.csv",
                        index_col=0,
                        parse_dates=[0]).reset_index()

In [11]:
# filtering for Community Areas only

covid = covid[covid['Geography Type']=="CA"]

In [12]:
# selecting and renaming relevant columns

covid = covid[['Community Area Name', 
               'Community Area or ZIP Code', 
               'CCVI Score', 
               'CCVI Category']].rename(columns={
                        "Community Area Name": "name",
                        'Community Area or ZIP Code': "comm_num",
                        "CCVI Score": "ccvi_score",
                        'CCVI Category': "ccvi_cat"})

In [13]:
# setting index to community area names to fix spelling

covid = covid.set_index(covid.columns[0]).rename(
    index={"Fuller Park*":"Fuller Park", "Burnside*":"Burnside"})

In [14]:
# reverting index to normal

covid=covid.reset_index()

In [15]:
# Chicago hardship index csv

hardship = pd.read_csv("../data/hardship_index.csv",
                        index_col=0,
                        parse_dates=[0]).reset_index()

In [16]:
# removing "Chicago" community area

hardship = hardship[hardship['COMMUNITY AREA NAME']!="CHICAGO"]

In [17]:
# selecting and renaming relevant columns

hardship = hardship[['COMMUNITY AREA NAME', 
               'HARDSHIP INDEX']].rename(columns={
                        "COMMUNITY AREA NAME": "name",
                        'HARDSHIP INDEX': "hardship_score"})

In [18]:
# setting index to community area names to fix spelling

hardship = hardship.set_index(covid.columns[0]).rename(
    index={"Montclaire":"Montclare", "Humboldt park":"Humboldt Park", "Washington Height":"Washington Heights"})

In [19]:
# reverting index to normal

hardship=hardship.reset_index()

In [20]:
# joining datasets

covid_hardship = covid.merge(hardship, on='name')

In [21]:
# 77 rows x 5 columns

covid_hardship.head(5)

,name,comm_num,ccvi_score,ccvi_cat,hardship_score
0,Ashburn,70,45.1,MEDIUM,37.0
1,Rogers Park,1,30.9,LOW,39.0
2,Lake View,6,5.2,LOW,5.0
3,Jefferson Park,11,25.6,LOW,25.0
4,Archer Heights,57,53.3,HIGH,67.0


##### Importing Chicago Community Area Census Tract Crosswalk

In [22]:
# importing Census tracts mapped to community area number 
# renaming columns

tracts = pd.read_csv("../data/tracts_comm_areas.csv",
                        index_col=0,
                        parse_dates=[0]).rename(columns={
                        "COMMAREA": "comm_num",          
                        "TRACTCE10": "tract"})

In [23]:
# importing community area numbers and names
# renaming columns

comm_area = pd.read_csv("../data/comm_areas.csv",
                        index_col=0,
                        parse_dates=[0]).rename(columns={
                        "AREA_NUMBE": "comm_num"})

In [24]:
# merging both dataframes above to map tract with community area name

tract_area= comm_area.merge(tracts, on='comm_num')

In [25]:
# selecting columns we need and renaming them

tract_area=tract_area[['comm_num', 'tract']]

In [26]:
# 801 rows x 3 columns
# final dataframe

tract_area.head(5)

,comm_num,tract
0,35,842000
1,35,351500
2,35,839500
3,35,839200
4,35,839600


##### Joining ACS, COVID index, Hardship Index, and Community Areas

In [27]:
# merging ACS with chicago community areas, covid index, hardship index

full_df = tract_area.merge(acs_df, on='tract'
                           ).merge(covid_hardship, on='comm_num')

In [28]:
# removing rows with no information

full_df = full_df[full_df['total_pop']!=0]

In [29]:
# 796 rows x 18 columns

full_df.sort_values(by="name").head(5)

,comm_num,tract,total_pop,hh_internet,hh_no_internet,hh_no_computer,total_households,hh_computer,hh_broadband,hh_median_income,hh_poverty(%),total_hispanic,total_black,hh_unemployment(%),total_white,total_asian,name,ccvi_score,ccvi_cat,hardship_score
108,14,140302,4189,1032,147,104,1292,1188,1032,75789,6.8,1433,228,5.6,1465,686,Albany Park,38.2,MEDIUM,53.0
103,14,140702,5882,1444,390,247,1997,1750,1444,57708,14.9,3746,183,5.8,1461,286,Albany Park,38.2,MEDIUM,53.0
104,14,140301,2839,579,165,89,812,723,579,50667,18.8,1724,57,7.5,397,642,Albany Park,38.2,MEDIUM,53.0
105,14,140601,2886,690,154,105,881,776,690,43988,12.1,1183,329,9.0,661,695,Albany Park,38.2,MEDIUM,53.0
106,14,140701,3028,898,140,75,1073,998,898,71125,11.0,1450,161,4.6,723,556,Albany Park,38.2,MEDIUM,53.0


##### Computer, Internet, Broadband Access

This section will look into the community areas at a household-level to see who has and who does not have basic access to the internet and/or computer. We will also look into the households who have a broadband internet subscription. 

In [30]:
# who has a computer? who has internet access? 
# selecting columns we need

internet_df = full_df[['name', 
                        'total_households',
                        'hh_no_internet', 
                        'hh_internet',
                        'hh_computer', 
                        'hh_no_computer',
                       'hh_broadband']].groupby(by="name").sum()

In [31]:
# calculating percentages of households for each variable

# percentage of households with/out internet access

internet_df['hh_no_internet(%)']=internet_df['hh_no_internet']/internet_df['total_households']*100
internet_df['hh_internet(%)']=internet_df['hh_internet']/internet_df['total_households']*100

# percentage of households with/out no computer

internet_df['hh_no_computer(%)']=internet_df['hh_no_computer']/internet_df['total_households']*100
internet_df['hh_computer(%)']=internet_df['hh_computer']/internet_df['total_households']*100

# percentage of households with/out broadband 

internet_df['hh_broadband(%)']=internet_df['hh_broadband']/internet_df['total_households']*100

In [32]:
# calculating response rates 

internet_df['internet_rr']=(internet_df['hh_internet']+internet_df['hh_no_internet'])/internet_df['total_households']*100
internet_df['computer_rr']=(internet_df['hh_computer']+internet_df['hh_no_computer'])/internet_df['total_households']*100


In [33]:
# resulting dataframe

internet_df.sort_values(["hh_internet(%)"],
                        ascending=False).head(5)

,total_households,hh_no_internet,hh_internet,hh_computer,hh_no_computer,hh_broadband,hh_no_internet(%),hh_internet(%),hh_no_computer(%),hh_computer(%),hh_broadband(%),internet_rr,computer_rr
name,,,,,,,,,,,,,
Lake View,53480,3065,48836,51552,1928,48705,5.731114,91.316380,3.605086,96.394914,91.071429,97.047494,100.0
Near South Side,14482,749,13204,13862,620,13204,5.171938,91.175252,4.281177,95.718823,91.175252,96.347190,100.0
Lincoln Park,32395,2026,29514,30706,1689,29489,6.254052,91.106652,5.213768,94.786232,91.029480,97.360704,100.0
North Center,14093,1161,12763,13419,674,12739,8.238132,90.562691,4.782516,95.217484,90.392393,98.800823,100.0
Lincoln Square,18347,1324,16505,17464,883,16488,7.216439,89.960211,4.812776,95.187224,89.867553,97.176650,100.0


Based on 2015-2019 ACS data, the percentages of households with computers, internet access, and internet broadband subscription are seen above. The neighborhoods of Burnside, Englewood, West Englewood, Fuller Park have the lowest percentages of both broadband subscription and internet access. The neighborhods of Near South Side, Lake View, Lincoln Park, and North Center have the highest. 

Broadband subscription and internet access numbers are extremely close, suggesting that the overwhelming majority of households who have access to internet do so via a broadbad subscription. 

##### Economics: Median Household Income, Poverty Rates, Unemployment Rates

In [34]:
# median household income, poverty rates, unemployment rates by community areas
# taking the median of ^ values  

income_df = full_df[['name', 
                      'hh_median_income', 
                      'hh_poverty(%)',
                    'hh_unemployment(%)']].groupby(by = "name").median().sort_values(["hh_unemployment(%)"], 
                                                                                  ascending = False)

income_df.head(5)

,hh_median_income,hh_poverty(%),hh_unemployment(%)
name,,,
Riverdale,15408.0,49.2,30.7
West Englewood,27277.0,29.9,26.1
Englewood,23125.0,36.6,25.7
Fuller Park,23746.5,21.7,25.3
Washington Park,27473.0,30.5,21.3


Median income, poverty rates, and unemployment rates are based on household-level data. The medians of median household income and poverty rates across tracts were used to make the table above. The median household incomes in Chicago community areas range from Riverdale's \$15,408 all the way to Lincoln Park's \$127,177. Riverdale also has the highest percentage of their households in poverty at 49.2%. Fuller Park has the highest unemployment rate at 57.6%.

##### Race & Ethnicity

In [35]:
# race and ethnicity by community area

race_df = full_df[['name', 
                        'total_pop',
                        'total_hispanic', 
                        'total_black',
                        'total_asian',
                        'total_white']].groupby(by = "name").sum().sort_values(["name"], 
                                                                      ascending = True)

In [36]:
# percentage of population hispanic

race_df['total_hispanic(%)']=race_df['total_hispanic']/race_df['total_pop']*100

# percentage of population black non-hispanic

race_df['total_black(%)']=race_df['total_black']/race_df['total_pop']*100

# percentage of population white non-hispanic

race_df['total_white(%)']=race_df['total_white']/race_df['total_pop']*100

# percentage of population asian non-hispanic

race_df['total_asian(%)']=race_df['total_asian']/race_df['total_pop']*100

In [37]:
# final race and ethnicity dataframe

race_df.sort_values(["total_asian(%)"], ascending = False).head(5)

,total_pop,total_hispanic,total_black,total_asian,total_white,total_hispanic(%),total_black(%),total_white(%),total_asian(%)
name,,,,,,,,,
Armour Square,13538,585,1135,9886,1878,4.321170,8.383809,13.872064,73.024080
Bridgeport,34540,7558,1039,13675,11825,21.881876,3.008107,34.235669,39.591778
North Park,18914,3807,523,4764,9128,20.127948,2.765148,48.260548,25.187692
McKinley Park,15177,8552,221,3625,2625,56.348422,1.456151,17.295908,23.884826
West Ridge,78466,14835,9086,18650,31836,18.906278,11.579538,40.572987,23.768256


Race and ethnicity data are based on total population numbers. Hispanic/Latino ethnicity was based on all races. Black/African-American race was non-Hispanic/Latino. Chicago community areas vary vastly in their race and ethnic compositions. Gage Park, South Lawndale, West Elsdon, and Hermosa have the highest percentages of Hispanics/Latinos of all races. Calumet Heights, Washington Heights, Avalon Park, and Oakland have the highest percentages of non-Hispanic/Latino African-American/Blacks. 

##### Internet Access & Demographics & Indices Combined

The table below shows all of the variables above.

In [38]:
# merging all tables by community area

final_df = internet_df.merge(income_df,
                        on='name').merge(race_df,
                                          on='name').merge(covid_hardship, on="name")

In [39]:
# reordering columns

final_df=final_df[['name', 
 'comm_num',
 'total_pop',
 'total_households', 
 'hh_no_internet', 
 'hh_no_internet(%)',
 'hh_internet',
 'hh_internet(%)',
 'internet_rr',
 'hh_computer',
 'hh_computer(%)',
 'hh_no_computer',
 'hh_no_computer(%)',
 'computer_rr',
 'hh_broadband',
 'hh_broadband(%)',
 'hh_poverty(%)',
 'hh_unemployment(%)',
 'hh_median_income',
 'total_hispanic',
 'total_hispanic(%)', 
 'total_black',  
 'total_black(%)', 
 'total_white',  
 'total_white(%)', 
 'total_asian',  
 'total_asian(%)', 
 'ccvi_score', 
 'ccvi_cat', 
 'hardship_score']]

In [40]:
final_df.sort_values(["hh_internet(%)"], ascending=True).head(5)


,name,comm_num,total_pop,total_households,hh_no_internet,hh_no_internet(%),hh_internet,hh_internet(%),internet_rr,hh_computer,...,total_hispanic(%),total_black,total_black(%),total_white,total_white(%),total_asian,total_asian(%),ccvi_score,ccvi_cat,hardship_score
12,Burnside,47,2006,888,375,42.229730,477,53.716216,95.945946,557,...,1.944167,1931,96.261216,10,0.498504,0,0.000000,58.4,HIGH,79.0
23,Englewood,68,22921,8983,3108,34.598686,5083,56.584660,91.183346,6728,...,3.690938,21692,94.638105,140,0.610794,68,0.296671,55.3,HIGH,94.0
70,West Englewood,67,26346,9483,3612,38.089212,5428,57.239270,95.328483,6670,...,9.367646,23097,87.667957,425,1.613148,24,0.091095,63.7,HIGH,89.0
25,Fuller Park,37,2330,1128,391,34.663121,666,59.042553,93.705674,783,...,5.107296,2160,92.703863,45,1.931330,0,0.000000,50.1,HIGH,97.0
52,North Lawndale,29,32073,11075,3321,29.986456,6631,59.873589,89.860045,8417,...,9.247654,27522,85.810495,1159,3.613631,49,0.152776,58.5,HIGH,87.0


In [41]:
final_df.sort_values(["total_black(%)"], ascending=False).head(15)


,name,comm_num,total_pop,total_households,hh_no_internet,hh_no_internet(%),hh_internet,hh_internet(%),internet_rr,hh_computer,...,total_hispanic(%),total_black,total_black(%),total_white,total_white(%),total_asian,total_asian(%),ccvi_score,ccvi_cat,hardship_score
6,Avalon Park,45,9713,4007,1030,25.705016,2840,70.875967,96.580983,3022,...,0.092659,9381,96.581901,93,0.957480,70,0.720684,44.2,MEDIUM,41.0
12,Burnside,47,2006,888,375,42.229730,477,53.716216,95.945946,557,...,1.944167,1931,96.261216,10,0.498504,0,0.000000,58.4,HIGH,79.0
67,Washington Heights,73,26742,9287,2129,22.924518,6712,72.273070,95.197588,8067,...,1.140528,25698,96.096029,360,1.346197,43,0.160796,53.2,HIGH,48.0
14,Chatham,44,30967,13907,4133,29.718847,9288,66.786510,96.505357,11298,...,0.846062,29625,95.666355,585,1.889108,124,0.400426,43.0,MEDIUM,60.0
29,Greater Grand Crossing,69,30149,12134,3840,31.646613,7700,63.458052,95.104665,9691,...,1.738034,28839,95.654914,304,1.008325,4,0.013267,46.0,MEDIUM,66.0
4,Auburn Gresham,71,45909,17161,5282,30.779092,10394,60.567566,91.346658,13724,...,2.180400,43791,95.386526,524,1.141388,141,0.307129,48.2,HIGH,74.0
61,Roseland,49,39162,14003,3648,26.051560,9942,70.999072,97.050632,11608,...,1.172055,37280,95.194321,557,1.422297,165,0.421327,54.7,HIGH,52.0
59,Riverdale,54,7359,2641,644,24.384703,1912,72.396819,96.781522,2218,...,3.505911,6990,94.985732,89,1.209403,22,0.298954,35.6,MEDIUM,98.0
23,Englewood,68,22921,8983,3108,34.598686,5083,56.584660,91.183346,6728,...,3.690938,21692,94.638105,140,0.610794,68,0.296671,55.3,HIGH,94.0
13,Calumet Heights,48,12815,5319,969,18.217710,4273,80.334649,98.552359,4455,...,3.722201,12056,94.077253,143,1.115880,9,0.070230,46.1,MEDIUM,38.0


In [42]:
final_df.sort_values(["hh_median_income"], ascending=True).head(5)

,name,comm_num,total_pop,total_households,hh_no_internet,hh_no_internet(%),hh_internet,hh_internet(%),internet_rr,hh_computer,...,total_hispanic(%),total_black,total_black(%),total_white,total_white(%),total_asian,total_asian(%),ccvi_score,ccvi_cat,hardship_score
59,Riverdale,54,7359,2641,644,24.384703,1912,72.396819,96.781522,2218,...,3.505911,6990,94.985732,89,1.209403,22,0.298954,35.6,MEDIUM,98.0
23,Englewood,68,22921,8983,3108,34.598686,5083,56.584660,91.183346,6728,...,3.690938,21692,94.638105,140,0.610794,68,0.296671,55.3,HIGH,94.0
25,Fuller Park,37,2330,1128,391,34.663121,666,59.042553,93.705674,783,...,5.107296,2160,92.703863,45,1.931330,0,0.000000,50.1,HIGH,97.0
71,West Garfield Park,26,16415,5353,1746,32.617224,3290,61.460863,94.078087,4047,...,2.954615,15276,93.061224,407,2.479440,0,0.000000,47.4,MEDIUM,92.0
17,Douglas,35,21353,9767,1776,18.183680,7751,79.359066,97.542746,8490,...,3.418723,14212,66.557392,2440,11.426966,3168,14.836323,36.0,MEDIUM,47.0


In [43]:
# export to Excel file in the data folder 

final_df.to_csv("../data/chicago_internet.csv") 

FileNotFoundError: [Errno 2] No such file or directory: 'data/chicago_internet.csv'

In [ ]:
final_df[['name', 
 'hh_internet(%)']].rename(columns={
    'name': "Community Area", "hh_internet(%)": "Households with Internet (%)"}).sort_values(
    ["Households with Internet (%)"], ascending=False).head(5)

***Variable explanations (26):***

Labels
+ name: Community Area Name
+ comm_num: Community Area Number (1 through 77)
+ total_pop: estimated total population
+ total_households: estimated total households

Internet-related
+ hh_no_internet: number of households with no internet access
+ hh_no_internet(%): percentage of households with no internet access
+ hh_internet: number of households with internet access
+ hh_internet(%): percentage of households with internet access
+ internet_rr: response rate for internet access 
+ hh_computer: number of households with a computer 
+ hh_computer(%): percentage of households with a computer 
+ hh_no_computer: number of households with no computer 
+ hh_no_computer(%): percentage of households with no computer 
+ computer_rr: response rate for computer access 
+ hh_broadband: number of households with an internet broadband subscription
+ hh_broadband(%): percent of households with an internet broadband subscription
 
Demographics
+ hh_poverty(%): household poverty rate
+ hh_unemployment(%): household unemployment rate
+ hh_median_income: household median income
+ total_hispanic': number of Hispanic/Latinos of ALL races
+ total_hispanic(%): percentage of Hispanic/Latinos of ALL races
+ total_black': number of non-Hispanic Black/African Americans
+ total_black(%)': percentage of non-Hispanic Black/African Americans
+ total_white': number of non-Hispanic Whites
+ total_white(%)': percentage of non-Hispanic Whites
+ total_asian': number of non-Hispanic Asians
+ total_asian(%)': percentage of non-Hispanic Asians
 
COVID & Hardship Indices
+ ccvi_score:  COVID-19 Community Vulnerability Index score
+ ccvi_cat':  COVID-19 Community Vulnerability Index score category
+ hardship_score: Score that incorporates six selected socioeconomic indicators (per capita income, education, age group, poverty level, unemployment, housing)

***Summary***

COMPUTER

Based on 2015-2019 ACS data, the percentages of households with computers, internet access, and internet broadband subscription are seen above. The neighborhoods of Burnside, Englewood, West Englewood, Fuller Park have the lowest percentages of both broadband subscription and internet access. The neighborhods of Near South Side, Lake View, Lincoln Park, and North Center have the highest. 
Broadband subscription and internet access numbers are extremely close, suggesting that the overwhelming majority of households who have access to internet do so via a broadbad subscription. 
  
  
ECONOMICS

The medians of median household income, poverty rates, and unemployment rates across tracts were used to make the table above. The median household incomes in Chicago community areas range from Riverdale's \\$15,408 all the way to Lincoln Park's \\$127,177. Riverdale also has the highest percentage of their households in poverty at 49.2%. Fuller Park has the highest unemployment rate at 57.6%.
  
  
RACE

Hispanic/Latino ethnicity was based on all races. Black/African-American race was non-Hispanic/Latino. Chicago community areas vary vastly in their race and ethnic compositions. Gage Park, South LawndalE, West Elsdon, and Hermosa have the highest percentages of Hispanics/Latinos of all races. Calumet Heights, Washington Heights, Avalon Park, and Oakland have the highest percentages of non-Hispanic/Latino African-American/Blacks. 
  
  
COVID VULNERABILITY

The community areas with the highest CCVI score were West Englewood, New City, and Gage Park. The community areas with the lowest scores also have the highest rates of internet access and median incomes. 
  
  
HARDSHIP

The community areas with the highest hardship scores were Riverdale, Fuller Park, and South Lawndale. Interestingly enough, Riverdale was labeled as Medium for the COVID Vulnerability score. 

***Discrepancies***

O'Hare:
The O'Hare population was way off in the table even after using the City of Chicago census tract community area crosswalk. The new numbers were 21,368 for total population with 9064 total households. However, the actual numbers are 13,418 for total population with 6,158 total households. 

After looking closely at the Census tracts within the Ohare community area, it was found that 3/6 census tracts were labeled as "partially outside the city boundary", where one of them was labeled as "partially outside the city boundary (almost all)". When excluding these tracts, the total population came to 12,861 with 6,120 households (~500 off from the 2021 population and 38 from the 2021 households). 

This is something we are in the works of addressing through spatial-based weighting: for all census tracts whose boundaries cross the higher-level geography’s boundaries (e.g., Chicago community area), estimate the proportion of the tract’s total area (e.g., square miles) that is located within each of the community areas it intersects. Use those proportions to assign ___% of the variable’s value to each community area. Of course, 100% of the variable’s value for a census tract that is fully subsumed within one community area is assigned to that community area.

### Correlations & Regressions

This section will look deeper into the relationships between variables through correlation coefficients and regression models. 

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(
    final_df[['hh_no_internet(%)','hh_median_income','hh_unemployment(%)','hh_poverty(%)','ccvi_score','hardship_score','total_hispanic(%)','total_black(%)']]
    .corr()
)